In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import callbacks
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,Dropout
import matplotlib.pyplot as plt
from sklearn import metrics
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

import keras
from keras import *
from keras.layers import *
from pythainlp import word_tokenize
from pythainlp.word_vector import *

from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

In [8]:
df = pd.read_csv('main_suicidal_data.csv').drop(columns='Unnamed: 0')

In [9]:
ws_tfidf = TfidfVectorizer(tokenizer=word_tokenize, ngram_range=(1, 2), sublinear_tf=True)
ws_vec = ws_tfidf.fit_transform(df['Tweet'])
y = df['Label (Specialist)']
X = ws_vec

In [10]:
le = LabelEncoder()
le.fit(y)
y = le.transform(y)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Random Forest Classifier

In [12]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier() 
clf.fit(X_train, y_train)
y_pred_rf_not_tuning = clf.predict(X_test)

In [13]:
print(classification_report(y_test, y_pred_rf_not_tuning))

              precision    recall  f1-score   support

           0       0.66      0.96      0.78       256
           1       0.44      0.23      0.30        48
           2       0.67      0.25      0.36        24
           3       0.56      0.22      0.32        41
           4       0.78      0.57      0.66        86
           5       0.81      0.44      0.57        59

    accuracy                           0.67       514
   macro avg       0.65      0.44      0.50       514
weighted avg       0.67      0.67      0.64       514



# HyperTuning Param

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in range(200,2000,200)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


In [ ]:
rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=0, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

In [ ]:
print(rf_random.best_estimator_)

In [ ]:
rf_tuned = RandomForestClassifier(bootstrap=False, max_depth=30, max_features='auto',
                       n_estimators=200)
rf_tuned.fit(X_train, y_train)
y_pred_rf_tuned = rf_tuned.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_rf_tuned))